#### Check session

In [1]:
# 텐서플로 버전을 확인합니다.
import tensorflow as tf
tf.__version__

'2.6.0'

In [2]:
# 텐서플로 설치 및 오류 방지를 위한 h5py 버전을 다운그레이드 합니다.
!pip install tensorflow-gpu==1.15
!pip install tensorflow-hub==0.7.0
!pip install 'h5py<3.0.0'

     |████████████████████████████████| 411.5 MB 6.8 kB/s 
     |████████████████████████████████| 503 kB 51.7 MB/s 
     |████████████████████████████████| 50 kB 8.6 MB/s 
     |████████████████████████████████| 3.8 MB 55.3 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=bc998b629778d8a10efa19b346f3f9258ea0529075f646dc91c7e2147395d5f9
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

     |████████████████████████████████| 89 kB 3.8 MB/s 
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.12.0
    Uninstalling tensorflow-hub-0.12.0:
      Successfully uninstalled tensorflow-hub-0.12.0
     |████████████████████████████████| 2.9 MB 4.1 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires gast==0.4.0, but you have gast 0.2.2 which is incompatible.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
tensorflow 2.6.0 requires tensorboard~=2.6, but you have tensorboard 1.15.0 which is incompatible.
tensorflow 2.6.0 requires tensorflow-estimator~=2.6, but you have tensorflow-estimator 1.15.1 which is incomp

#### Restart runtime

#### Mount

In [1]:
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


#### Check directories and Change directory

In [2]:
!ls

drive  sample_data


In [3]:
# 각자에게 맞는 directory를 설정해주세요.
eojeol = "/content/drive/MyDrive/bert_classification_kor/eojeol"
os.chdir(eojeol)

#### Check the current version of tf

In [4]:
# tf 버전 확인
import tensorflow as tf
print(tf.__version__)

1.15.0


In [5]:
!ls

 004_bert_eojeol_tensorflow     readers
 bert-module		        requirements_colab.txt
 bert_to_keras.py	        run_intent_classify.py
 best_model.h5		        sample_data
 eval_bert_finetuning.py        sample_data.txt
 final_test_1.txt	        server.py
 final_test.gdoc	        settings.py
 final_test.txt		        static
 final_train_1.txt	        templates
 final_train.gdoc	        test_1.txt
 final_train.txt	        test_data
 final_validation_1.txt         test_model
 final_validation.txt	        train_1.txt
 hello_BERT		        train_bert_finetuning.py
 inference.ipynb	        train_data
 inference.py		        train_log.png
 KorBERT_classification.ipynb   train_model
 korbert_demo.ipynb	        utils.py
 layers			        validation_1.txt
 modeling.py		        validation_data
 models			        vectorizers
'predict값 입력하기.ipynb'      with_place_refined.csv
 prepare_data.py	        with_place_revised_pred.csv
 __pycache__


In [6]:
!nvidia-smi
!export CUDA_VISIBLE_DEVICES=0

Tue Nov  9 00:09:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Model load and predict

In [11]:
# pred_text 정의

from vectorizers.bert_vectorizer import BERTVectorizer
from vectorizers import tokenizationK
from models.bert_nlu_model import BertNLUModel
from utils import flatten

import argparse
import os
import pickle
import tensorflow as tf

load_folder_path = 'test_model' # 모델 경로 입력
type_ = 'bert'

# this line is to disable gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

config = tf.ConfigProto(intra_op_parallelism_threads=1,
                        inter_op_parallelism_threads=1,
                        allow_soft_placement=True,
                        device_count = {'CPU': 1})
sess = tf.compat.v1.Session(config=config)

if type_ == 'bert':
    bert_model_hub_path = './bert-module'
    is_bert = True
elif type_ == 'albert':
    bert_model_hub_path = 'https://tfhub.dev/google/albert_base/1'
    is_bert = False
else:
    raise ValueError('type must be one of these values: %s' % str(VALID_TYPES))


bert_vectorizer = BERTVectorizer(is_bert, bert_model_hub_path)

# loading models
print('Loading models ...')
if not os.path.exists(load_folder_path):
    print('Folder `%s` not exist' % load_folder_path)

with open(os.path.join(load_folder_path, 'intents_label_encoder.pkl'), 'rb') as handle:
    intents_label_encoder = pickle.load(handle)
    intents_num = len(intents_label_encoder.classes_)


model = BertNLUModel.load(load_folder_path, sess)

vocab_file = "./bert-module/assets/vocab.korean.rawtext.list"
tokenizer = tokenizationK.FullTokenizer(vocab_file=vocab_file)

def pred_text(x):
    input_text = x.strip()
    tokens = tokenizer.tokenize(input_text)
    tokens = [' '.join(tokens)]

    data_input_ids, data_input_mask, data_segment_ids = bert_vectorizer.transform(tokens)
    
    first_inferred_intent, first_inferred_intent_score, _, _ = model.predict_intent([data_input_ids, data_input_mask, data_segment_ids], intents_label_encoder)
    return [first_inferred_intent, first_inferred_intent_score]

initializing!
is_bert : True
bert_model_hub_path : ./bert-module
initialized!
Loading models ...
bert inputs : [<tf.Tensor 'input_ids_2:0' shape=(?, ?) dtype=int32>, <tf.Tensor 'input_masks_2:0' shape=(?, ?) dtype=int32>, <tf.Tensor 'segment_ids_2:0' shape=(?, ?) dtype=int32>]
init ok
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
call ok
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
segmen

In [ ]:
import pandas as pd

df = pd.read_csv('with_place_refined.csv') # 데이터프레임 불러오기
df.head()

,score,comment,place
0,5.0,코로나로 인해 사람이 인적이 적어졌지만 비오늘날 서울 시티트어를 하기엔 너무 좋은 ...,N서울타워
1,5.0,서울의 전경을 볼 수 있어 너무 좋아요. 서두르세요. 다음엔 남산타워 전망대 티켓까...,N서울타워
2,5.0,"분위기 전환, 데이트하러 방문하기 좋은 거 같아요. 비오는 날이라 아쉽지만 그래도 ...",N서울타워
3,4.0,서울에 사는 사람들에게는 너무 자주 볼 수 있어서 막상 살면서 몇번 안 올라가게 되...,N서울타워
4,4.0,필수코스. 나 자신은 서울사람으로 여러번 다녀왔지만 아기와 남편을 위해 가 봄. 서...,N서울타워


In [ ]:
df[['prediction']] = df.comment.apply(pred_text)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform started
transform 

In [ ]:
df.iloc[15151]

score                            5
comment            국에 대한 좋은 정보가 가득
place         국립고궁박물관
prediction      [[2], [0.9998166]]
Name: 15151, dtype: object

In [ ]:
df['emotion'] = df.prediction.apply(lambda x: x[0][0])
df['pred_score'] = df.prediction.apply(lambda x: x[1][0])

In [ ]:
df = df.drop('prediction', axis = 1)
df.to_csv('with_place_revised_pred.csv', index = False)